In [11]:
import os 

In [12]:
%pwd

'd:\\sign_language_conversion'

In [13]:
os.chdir("../")

In [14]:
%pwd

'd:\\'

In [15]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataCreation:
    root_dir: Path
    local_data_file: Path
    number_of_classes: int

In [16]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

In [17]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_creation(self) -> DataCreation:
        config = self.config.data_creation
        create_directories([config.root_dir])

        # Check if 'no_of_classes' exists in params
        if 'no_of_classes' not in self.params:
            raise KeyError("The key 'no_of_classes' is missing from params file.")

        data_creation_config = DataCreation(
            root_dir=config.root_dir,
            local_data_file=config.raw_dataset,
            number_of_classes=self.params['no_of_classes']
        )
        return data_creation_config


In [18]:
import os
import cv2
from sign_language_conversion import logger
from sign_language_conversion.utils.common import get_size

In [19]:
class Data_Creation:
    def __init__(self, config: DataCreation):
        self.config = config
        
    def capture_data(self):
        DATA_DIR = self.config.local_data_file
        number_of_classes = self.config.number_of_classes
        dataset_size = 100

        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)

        def find_working_camera_index():
            for index in range(5):
                cap = cv2.VideoCapture(index)
                if cap.isOpened():
                    cap.release()
                    return index
            return None

        camera_index = find_working_camera_index()
        if camera_index is None:
            print("Error: Could not find a working camera.")
            return

        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            print("Error: Could not open video device.")
            return

        for j in range(number_of_classes):
            class_dir = os.path.join(DATA_DIR, str(j))
            if not os.path.exists(class_dir):
                os.makedirs(class_dir)

            print(f'Collecting data for class {j}')

            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Error: Failed to capture image.")
                    break

                cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
                cv2.imshow('frame', frame)
                if cv2.waitKey(25) == ord('q'):
                    break

            counter = 0
            while counter < dataset_size:
                ret, frame = cap.read()
                if not ret:
                    print("Error: Failed to capture image.")
                    break

                cv2.imshow('frame', frame)
                cv2.waitKey(20)
                cv2.imwrite(os.path.join(class_dir, f'{counter}.jpg'), frame)
                counter += 1

                if cv2.waitKey(1) & 0xFF == ord('s'):
                    break

        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(5)

In [20]:
try:
    config = ConfigurationManager()
    data_creation_config = config.get_creation()
    data_creator = Data_Creation(config=data_creation_config)
    data_creator.capture_data()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'